In [7]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from scipy.stats import zscore
import math
import os

In [8]:
# df["zone"] = df.apply(lambda row: "thermal" if (
#         row["climb_m(delta)"] > 0 or
#         row["climb_rate_m/s"] > 0)
#         else "standart", axis=1)

df = pd.read_csv(os.path.join("data", "flight_data.csv"), low_memory=False)

In [9]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2254810 entries, 0 to 2254809
Data columns (total 25 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   filename                 object 
 1   datetime                 object 
 2   pilot                    object 
 3   latitude                 float64
 4   longitude                float64
 5   gps_altitude_m           int64  
 6   pressure_altitude_m      int64  
 7   previous_latitude        float64
 8   previous_longitude       float64
 9   distance_from_takeoff_m  float64
 10  distance_m               float64
 11  speed_km/s               float64
 12  climb_m                  float64
 13  climb_m(delta)           float64
 14  climb_rate_m/s           float64
 15  bearing                  int64  
 16  delta_bearing            float64
 17  glide_ratio              float64
 18  elapsed_time             float64
 19  temp                     float64
 20  pressure                 float64
 21  humidity

,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,distance_m,speed_km/s,climb_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,...,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06,2.254810e+06
mean,3.841927e+01,3.203616e+01,2.161857e+03,2.624519e+03,3.841930e+01,3.203611e+01,1.211919e+05,1.616479e+01,4.965048e+01,-1.165109e-02,...,1.437307e+02,1.073991e+01,5.715142e+00,1.363105e+04,3.082639e+01,1.007536e+03,2.673022e+01,8.941817e+00,3.180068e+00,2.201502e+02
std,1.059887e+00,2.652021e+00,2.052659e+03,7.312319e+02,1.059917e+00,2.651968e+00,8.787638e+04,1.663380e+02,3.106814e+01,5.968674e+00,...,9.253008e+01,1.599066e+01,8.022626e+00,8.083090e+03,3.579166e+00,2.622456e+00,7.263755e+00,3.133182e+00,1.370671e+00,1.167001e+02
min,3.650395e+01,2.463248e+01,-7.347000e+03,0.000000e+00,3.650395e+01,2.463248e+01,0.000000e+00,0.000000e+00,-5.680640e+01,-6.668000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.838000e+01,9.990000e+02,6.000000e+00,-5.280000e+00,0.000000e+00,0.000000e+00
25%,3.743282e+01,3.010410e+01,9.730000e+02,2.114000e+03,3.743278e+01,3.010407e+01,4.529556e+04,1.119458e+01,3.951597e+01,-1.000000e+00,...,6.100000e+01,2.000000e+00,0.000000e+00,6.712000e+03,2.845000e+01,1.006000e+03,2.200000e+01,6.960000e+00,2.140000e+00,1.660000e+02
50%,3.836102e+01,3.224477e+01,2.250000e+03,2.615000e+03,3.836105e+01,3.224480e+01,1.079542e+05,1.465554e+01,5.109569e+01,0.000000e+00,...,1.470000e+02,6.000000e+00,3.983470e+00,1.344900e+04,3.055000e+01,1.007000e+03,2.700000e+01,9.220000e+00,3.160000e+00,2.480000e+02
75%,3.918322e+01,3.269483e+01,2.859000e+03,3.120000e+03,3.918328e+01,3.269480e+01,1.925523e+05,1.722093e+01,6.086057e+01,1.000000e+00,...,2.030000e+02,1.300000e+01,9.239974e+00,2.016800e+04,3.318000e+01,1.009000e+03,3.200000e+01,1.111000e+01,4.060000e+00,3.200000e+02
max,4.272443e+01,4.104910e+01,1.619900e+04,9.999000e+03,4.272443e+01,4.104910e+01,3.681782e+05,2.484829e+05,3.084615e+04,1.428000e+03,...,3.590000e+02,1.800000e+02,5.176727e+03,3.667000e+04,4.111000e+01,1.018000e+03,5.800000e+01,2.225000e+01,8.700000e+00,3.600000e+02


In [10]:
def zscore_calc(df):
    column_names = df.columns.to_list()
    for column_name in column_names:
        df["zscore_" + column_name] = zscore(df[column_name])
    column_names = [name for name in df.columns if name.startswith("zscore")]
    def findzscore(row):
        for column_name in column_names:
            if abs(row[column_name]) > 3:
                return True
        return False
    # df["deleted"] = df.apply(findzscore, axis=1)
    return df.apply(findzscore, axis=1)
    

# "gps_altitude_m", "pressure_altitude_m", "distance_from_takeoff_m", "distance_m", "speed_km/s", "climb_m", "climb_m(delta)", "climb_rate_m/s", "glide_ratio"
df["zscore"] = zscore_calc(df[["gps_altitude_m", "pressure_altitude_m", "distance_from_takeoff_m", "distance_m", "speed_km/s", "climb_m", "climb_m(delta)", "climb_rate_m/s", "glide_ratio"]].copy())

# df["gps_altitude_m"] = np.where(
#     (df["gps_altitude_m"] < 0) | (abs(zscore(df["gps_altitude_m"])) > 3),
#     df["pressure_altitude_m"],
#     df["gps_altitude_m"]
# )
# df["pressure_altitude_m"] = np.where(
#     (df["pressure_altitude_m"] < 0) | (abs(zscore(df["pressure_altitude_m"])) > 3),
#     df["gps_altitude_m"],
#     df["pressure_altitude_m"]
# )

# df.drop(["gpszscore", "presurezscore"], axis=1, inplace=True)
df = df[~(((df["gps_altitude_m"] == 0) & (df["pressure_altitude_m"] == 0)) | (df["zscore"] == True))]
df.drop("zscore", axis=1, inplace=True)
df.head()

,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,file_8584_2011-07-12-XXX000-00-YURDAER-ETIKE.igc,2011-07-12 09:42:41,11000706,39.522883,30.849200,1244,1286,39.522883,30.849117,17.296307,...,89,0.0,2.388478,425.0,26.61,1010.0,45.0,13.71,3.47,24.0
1,file_8584_2011-07-12-XXX000-00-YURDAER-ETIKE.igc,2011-07-12 09:42:42,11000706,39.522883,30.849267,1247,1286,39.522883,30.849200,23.003930,...,89,0.0,1.910782,426.0,26.61,1010.0,45.0,13.71,3.47,24.0
2,file_8584_2011-07-12-XXX000-00-YURDAER-ETIKE.igc,2011-07-12 09:42:43,11000706,39.522867,30.849333,1250,1286,39.522883,30.849267,28.899678,...,107,18.0,2.007870,427.0,26.61,1010.0,45.0,13.71,3.47,24.0
3,file_8584_2011-07-12-XXX000-00-YURDAER-ETIKE.igc,2011-07-12 09:42:44,11000706,39.522833,30.849367,1250,1286,39.522867,30.849333,32.385100,...,142,35.0,0.000000,428.0,26.61,1010.0,45.0,13.71,3.47,24.0
4,file_8584_2011-07-12-XXX000-00-YURDAER-ETIKE.igc,2011-07-12 09:42:45,11000706,39.522800,30.849383,1250,1287,39.522833,30.849367,34.780668,...,158,16.0,0.000000,429.0,26.61,1010.0,45.0,13.71,3.47,24.0


In [11]:
df.describe()

,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,distance_m,speed_km/s,climb_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,...,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06,2.155407e+06
mean,3.839850e+01,3.197466e+01,1.862786e+03,2.637239e+03,3.839852e+01,3.197462e+01,1.213395e+05,1.525904e+01,4.951577e+01,9.059078e-03,...,1.432513e+02,1.048238e+01,5.558014e+00,1.369663e+04,3.091596e+01,1.007415e+03,2.662347e+01,8.952603e+00,3.122870e+00,2.182633e+02
std,1.064534e+00,2.663057e+00,1.239355e+03,7.329378e+02,1.064563e+00,2.663014e+00,8.833986e+04,1.122784e+01,1.590578e+01,1.910471e+00,...,9.354057e+01,1.493455e+01,6.142485e+00,8.120996e+03,3.544804e+00,2.502055e+00,7.292823e+00,3.124954e+00,1.356507e+00,1.161070e+02
min,3.650395e+01,2.463248e+01,0.000000e+00,4.310000e+02,3.650395e+01,2.463248e+01,0.000000e+00,0.000000e+00,-0.000000e+00,-1.700000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.838000e+01,9.990000e+02,6.000000e+00,-5.280000e+00,0.000000e+00,0.000000e+00
25%,3.741825e+01,3.003627e+01,0.000000e+00,2.121000e+03,3.741820e+01,3.003620e+01,4.497540e+04,1.111425e+01,3.951169e+01,-1.000000e+00,...,5.700000e+01,2.000000e+00,0.000000e+00,6.739000e+03,2.854000e+01,1.006000e+03,2.200000e+01,6.980000e+00,2.110000e+00,1.640000e+02
50%,3.832673e+01,3.198903e+01,2.203000e+03,2.628000e+03,3.832673e+01,3.198910e+01,1.075909e+05,1.427542e+01,5.101216e+01,0.000000e+00,...,1.460000e+02,6.000000e+00,4.055429e+00,1.352300e+04,3.063000e+01,1.007000e+03,2.600000e+01,9.220000e+00,3.100000e+00,2.430000e+02
75%,3.914962e+01,3.265672e+01,2.801000e+03,3.140000e+03,3.914967e+01,3.265668e+01,1.932873e+05,1.720046e+01,6.084706e+01,1.000000e+00,...,2.040000e+02,1.300000e+01,9.278268e+00,2.028800e+04,3.324000e+01,1.009000e+03,3.200000e+01,1.111000e+01,3.980000e+00,3.190000e+02
max,4.272443e+01,4.104910e+01,4.609000e+03,4.818000e+03,4.272443e+01,4.104910e+01,3.681782e+05,3.747842e+02,1.428325e+02,1.700000e+01,...,3.590000e+02,1.800000e+02,2.977806e+01,3.667000e+04,4.111000e+01,1.018000e+03,5.800000e+01,2.225000e+01,8.700000e+00,3.600000e+02


In [12]:
df = df[df["filename"] == "file_17415_2013-08-04_Basat_Ayas_251km.igc"]
df.head()

,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
1833032,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:06,Basat,40.032200,32.326233,1048,779,40.032250,32.326267,413.117198,...,207,13.0,6.238333,15.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833033,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:07,Basat,40.032233,32.326167,1048,792,40.032200,32.326233,419.833207,...,303,96.0,0.000000,16.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833034,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:08,Basat,40.032300,32.326133,1047,799,40.032233,32.326167,425.552413,...,339,36.0,7.930290,17.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833035,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:09,Basat,40.032350,32.326117,1048,806,40.032300,32.326133,429.248951,...,345,6.0,5.731121,18.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833036,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:10,Basat,40.032400,32.326083,1047,815,40.032350,32.326117,434.261542,...,332,13.0,6.238330,19.0,26.71,1013.0,29.0,7.22,2.79,354.0
